## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-13-21-00-15 +0000,nyt,White House Sees Win After 2 Strong Economic R...,https://www.nytimes.com/2026/02/13/business/ec...
1,2026-02-13-20-59-51 +0000,nypost,"Dozens of students, families evicted from NYC ...",https://nypost.com/2026/02/13/us-news/dozens-o...
2,2026-02-13-20-59-04 +0000,cbc,Former minister Freeland violated election rul...,https://www.cbc.ca/news/politics/chrystia-free...
3,2026-02-13-20-56-52 +0000,nypost,Luxe grocer Erewhon ups its med spa game with ...,https://nypost.com/2026/02/13/lifestyle/erewho...
4,2026-02-13-20-56-12 +0000,nyt,Live Updates: Macron Insists Europe Remains Ce...,https://www.nytimes.com/live/2026/02/13/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2515/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
69,trump,36
410,epstein,21
299,new,14
111,ice,12
518,world,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
366,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...,97
209,2026-02-13-12-00-00 +0000,wsj,Republican Sen. Rand Paul joined Democrats in ...,https://www.wsj.com/politics/gop-sen-rand-paul...,87
250,2026-02-13-10-00-16 +0000,wapo,Services cut. Staff laid off. New Trump cuts h...,https://www.washingtonpost.com/education/2026/...,79
304,2026-02-13-02-17-00 +0000,nypost,Minnesota Gov. Tim Walz slammed over proposed ...,https://nypost.com/2026/02/12/us-news/minnesot...,69
339,2026-02-12-23-16-31 +0000,nypost,Here’s the real reason Goldman CEO David Solom...,https://nypost.com/2026/02/12/business/heres-t...,69


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
366,97,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...
339,69,2026-02-12-23-16-31 +0000,nypost,Here’s the real reason Goldman CEO David Solom...,https://nypost.com/2026/02/12/business/heres-t...
107,64,2026-02-13-17-39-00 +0000,wsj,"Chris Klomp, the head of Medicare and a former...",https://www.wsj.com/politics/policy/white-hous...
136,56,2026-02-13-16-18-00 +0000,wsj,What a shutdown would mean for the Department ...,https://www.wsj.com/politics/policy/homeland-s...
193,50,2026-02-13-13-04-33 +0000,nypost,Trans shooter Jesse Van Rootselaar — who kille...,https://nypost.com/2026/02/13/world-news/jesse...
63,43,2026-02-13-19-08-52 +0000,nypost,San Francisco schools chief ripped for ‘crocod...,https://nypost.com/2026/02/13/us-news/san-fran...
333,40,2026-02-12-23-33-00 +0000,wsj,When Customs and Border Protection fired a las...,https://www.wsj.com/politics/national-security...
6,40,2026-02-13-20-53-59 +0000,bbc,Ex-CNN host Don Lemon pleads not guilty in Min...,https://www.bbc.com/news/articles/cge89r1e158o...
83,37,2026-02-13-18-35-50 +0000,nyt,"Amid Fallout From Epstein Files, Dubai’s DP Wo...",https://www.nytimes.com/2026/02/13/world/middl...
84,32,2026-02-13-18-35-08 +0000,nypost,Amazon’s Ring axes deal with police tech provi...,https://nypost.com/2026/02/13/business/amazons...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
